In [4]:
import pandas as pd
import numpy as np

# Create a sample DataFrame
data = {
    'Name': ['John', 'Anna', 'Peter', 'Linda', 'Sarah', 'Michael', 'Emma', 'David'],
    'Age': [28, 34, 29, 42, 31, 35, 27, 38],
    'City': ['New York', 'Paris', 'Berlin', 'London', 'Sydney', 'New York', 'Paris', 'London'],
    'Department': ['HR', 'IT', 'Sales', 'Marketing', 'Finance', 'IT', 'HR', 'Sales'],
    'Salary': [65000, 70000, 62000, 85000, 72000, 78000, 63000, 69000],
    # Using 'ME' (month end) instead of 'M' to avoid deprecation warning
    'JoinDate': pd.date_range('2020-01-01', periods=8, freq='ME'),
    'Gender': ['M', 'F', 'M', 'F', 'F', 'M', 'F', 'M']
}
df = pd.DataFrame(data)

# Display original DataFrame
print("Original DataFrame:")
print(df)

# Basic grouping operations
dept_group = df.groupby('Department')
print("\nGroups in 'Department':")
print(sorted(list(dept_group.groups.keys())))

print("\nEmployees in IT department:")
print(dept_group.get_group('IT'))

# Basic aggregations
print("\nMean salary by department:")
print(dept_group['Salary'].mean())

print("\nMultiple statistics by department:")
print(dept_group['Salary'].agg(['min', 'max', 'mean', 'std']))

# Multiple column grouping
multi_group = df.groupby(['Department', 'Gender'])
print("\nMean salary by department and gender:")
print(multi_group['Salary'].mean())

# Multiple aggregations for different columns
agg_dict = {'Salary': ['min', 'max', 'mean'], 'Age': ['min', 'max', 'mean']}
print("\nMultiple aggregations by department:")
print(dept_group.agg(agg_dict))

# Named aggregations
named_agg = dept_group.agg(
    min_salary=('Salary', 'min'),
    max_salary=('Salary', 'max'),
    avg_salary=('Salary', 'mean'),
    min_age=('Age', 'min'),
    max_age=('Age', 'max'),
    avg_age=('Age', 'mean')
)
print("\nNamed aggregations by department:")
print(named_agg)

# Custom aggregation function
def salary_range(x):
    return x.max() - x.min()

print("\nCustom aggregation (salary range) by department:")
print(dept_group['Salary'].agg(salary_range))

print("\nMultiple custom and built-in aggregations:")
print(dept_group['Salary'].agg([salary_range, 'mean', 'std']))

# Transformations
print("\nStandardized salary within each department:")
print(dept_group['Salary'].transform(lambda x: (x - x.mean()) / x.std()))

print("\nPercentage of department's total salary:")
print(dept_group['Salary'].transform(lambda x: x / x.sum() * 100))

# Filtering
high_salary_depts = dept_group.filter(lambda x: x['Salary'].mean() > 70000)
print("\nDepartments with average salary > 70000:")
print(high_salary_depts)

large_depts = dept_group.filter(lambda x: len(x) > 1)
print("\nDepartments with more than 1 employee:")
print(large_depts)

# Apply function to groups
print("\nSalary summary by department:")
print(dept_group['Salary'].apply(lambda x: pd.Series({
    'min': x.min(),
    'max': x.max(),
    'range': x.max() - x.min(),
    'mean': x.mean(),
    'median': x.median()
})))

# Time-based grouping
df['JoinYear'] = df['JoinDate'].dt.year
year_group = df.groupby('JoinYear')
print("\nAverage salary by join year:")
print(year_group['Salary'].mean())

df['JoinMonth'] = df['JoinDate'].dt.month
month_group = df.groupby('JoinMonth')
print("\nAverage salary by join month:")
print(month_group['Salary'].mean())

# Pivot tables
pivot = pd.pivot_table(df, 
                      values='Salary', 
                      index='Department', 
                      columns='Gender', 
                      aggfunc='mean')
print("\nPivot table of average salary by department and gender:")
print(pivot)

# Multi-value pivot table
multi_pivot = pd.pivot_table(
    df, 
    values=['Salary', 'Age'], 
    index=['Department'], 
    columns=['Gender'], 
    aggfunc={'Salary': 'mean', 'Age': 'mean'},
    margins=True,
    margins_name='All'
)
print("\nPivot table with multiple values and aggregations:")
print(multi_pivot)

# Crosstab
cross = pd.crosstab(
    df['Department'], 
    df['Gender'], 
    values=df['Salary'], 
    aggfunc='mean'
)
print("\nCrosstab of average salary by department and gender:")
print(cross)

# Hierarchical indexing
# Adding a unique identifier to avoid duplicate index
df['Employee_ID'] = range(len(df))
df_hier = df.set_index(['Department', 'Gender', 'Employee_ID'])
print("\nDataFrame with hierarchical index:")
print(df_hier)

print("\nMean salary by department level:")
print(df_hier.groupby(level=0)['Salary'].mean())

print("\nMean salary by gender level:")
print(df_hier.groupby(level=1)['Salary'].mean())

# Categorical grouping
df['Salary_Category'] = pd.cut(
    df['Salary'], 
    bins=[0, 65000, 75000, 100000], 
    labels=['Low', 'Medium', 'High']
)
# Add observed=True to fix the warning
cat_group = df.groupby(['Department', 'Salary_Category'], observed=True)
print("\nCount of employees by department and salary category:")
print(cat_group.size().unstack(fill_value=0))

# Time series analysis
df_sorted = df.sort_values('JoinDate')
print("\nRolling average of last 3 hires' salaries:")
print(df_sorted['Salary'].rolling(window=3).mean())

print("\nCumulative average salary:")
print(df_sorted['Salary'].expanding().mean())

# Time series resampling
ts = pd.Series(df['Salary'].values, index=df['JoinDate'])
print("\nMonthly salary time series:")
print(ts)

# Change 'Q' to 'QE' to fix the warning
quarterly = ts.resample('QE').mean()
print("\nQuarterly average salary:")
print(quarterly)

# Ranking and percentiles
df['Salary_Rank'] = df.groupby('Department')['Salary'].transform(
    lambda x: x.rank(method='min', ascending=False)
)
print("\nSalary rank within department (1 is highest):")
print(df[['Name', 'Department', 'Salary', 'Salary_Rank']])

df['Salary_Percentile'] = df.groupby('Department')['Salary'].transform(
    lambda x: x.rank(pct=True) * 100
)
print("\nSalary percentile within department:")
print(df[['Name', 'Department', 'Salary', 'Salary_Percentile']])

# Cumulative statistics
df_sorted = df.sort_values(['Department', 'JoinDate'])
df_sorted['Cumulative_Dept_Salary'] = df_sorted.groupby('Department')['Salary'].cumsum()
print("\nCumulative salary by department (ordered by join date):")
print(df_sorted[['Name', 'Department', 'JoinDate', 'Salary', 'Cumulative_Dept_Salary']])

Original DataFrame:
      Name  Age      City Department  Salary   JoinDate Gender
0     John   28  New York         HR   65000 2020-01-31      M
1     Anna   34     Paris         IT   70000 2020-02-29      F
2    Peter   29    Berlin      Sales   62000 2020-03-31      M
3    Linda   42    London  Marketing   85000 2020-04-30      F
4    Sarah   31    Sydney    Finance   72000 2020-05-31      F
5  Michael   35  New York         IT   78000 2020-06-30      M
6     Emma   27     Paris         HR   63000 2020-07-31      F
7    David   38    London      Sales   69000 2020-08-31      M

Groups in 'Department':
['Finance', 'HR', 'IT', 'Marketing', 'Sales']

Employees in IT department:
      Name  Age      City Department  Salary   JoinDate Gender
1     Anna   34     Paris         IT   70000 2020-02-29      F
5  Michael   35  New York         IT   78000 2020-06-30      M

Mean salary by department:
Department
Finance      72000.0
HR           64000.0
IT           74000.0
Marketing    85000.0
S